# 1. Import Required Libraries

In [ ]:
import json
import boto3
import sagemaker

from sagemaker import ModelPackage
from sagemaker import get_execution_role

# 2. Define Model Configuration and Regional ARNs

In [ ]:
MODEL_PACKAGE_NAME = "reranker-0630-1763528603-2cff7de9bd9c380eb990e02b57a4ceb3"

MODEL_PACKAGE_MAP = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{MODEL_PACKAGE_NAME}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{MODEL_PACKAGE_NAME}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{MODEL_PACKAGE_NAME}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{MODEL_PACKAGE_NAME}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{MODEL_PACKAGE_NAME}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{MODEL_PACKAGE_NAME}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{MODEL_PACKAGE_NAME}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{MODEL_PACKAGE_NAME}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{MODEL_PACKAGE_NAME}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{MODEL_PACKAGE_NAME}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{MODEL_PACKAGE_NAME}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{MODEL_PACKAGE_NAME}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{MODEL_PACKAGE_NAME}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{MODEL_PACKAGE_NAME}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{MODEL_PACKAGE_NAME}",
}

MODEL_NAME = "ixi-DocReranker-v1"

# 3. Initialize Session and Validate Region

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

if region not in MODEL_PACKAGE_MAP.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

# 4. Load and Deploy Model from Marketplace

In [ ]:
model_pack_arn = MODEL_PACKAGE_MAP[region]
model = ModelPackage(
    role=role,
    model_package_arn=model_pack_arn,
    sagemaker_session=sagemaker_session
)

print(f"role: {role}")
print(f"region: {region}")
print(f"model package arn: {model_pack_arn}")

model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=sagemaker.utils.name_from_base(MODEL_NAME),
    model_data_download_timeout=600,
    container_startup_health_check_timeout=300,
)

print(f"endpoint name: {model.endpoint_name}")

# 5. Run Inference on Deployed Endpoint

In [ ]:
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=region)

prompt = "who are you?"

request_data = {
    "query": "수면의 질을 높이는 방법",
    "documents": [
        "규칙적인 수면 시간은 깊은 잠을 돕는다.",
        "조용하고 어두운 환경은 숙면에 도움이 된다.",
        "고양이는 따뜻한 곳을 좋아한다.",
        "가벼운 스트레칭은 몸을 이완시켜 잘 잘 수 있게 한다."
    ]
}


response = sagemaker_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(request_data),
)

result = response['Body'].read().decode('utf-8')
print("response:", result)

# 6.Clean Up Resources

In [ ]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)
model.delete_model()